In [ ]:
#pip install vaderSentiment

In [ ]:
#pip install pattern


In [1]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import pandas as pd
import numpy as np
from scipy.sparse import hstack
import spacy
import re
import nltk
from nltk.tokenize import word_tokenize
nlp = spacy.load("en_core_web_sm")
nlp.Defaults.stop_words -= {"not", "do"}
from autocorrect import Speller
nltk.download('words', quiet=True)
import emoji
import sqlite3
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from textblob import TextBlob
from nltk.corpus import words
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, ConfusionMatrixDisplay
from sklearn.model_selection import RandomizedSearchCV, train_test_split
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
import contractions
nltk.download('wordnet', quiet=True)
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import gensim
import gensim.corpora as corpora
from gensim.models import CoherenceModel
from gensim.utils import simple_preprocess
from gensim.models.ldamodel import LdaModel
from pprint import pprint
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.decomposition import PCA
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, ConfusionMatrixDisplay
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from scipy.stats import randint
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn import svm


# Import dataset

In [4]:
conn = sqlite3.connect("IMDB_Movies_2021.db")

cursor = conn.cursor()

table_name = "REVIEWS"

df = pd.read_sql_query(f"SELECT * FROM {table_name}", conn)

conn.close()

# Drop duplicates and remove nulls

In [5]:
df = df.drop_duplicates(subset = ['REVIEW','AUTHOR'])
df = df.dropna()

# Combine review and title

In [6]:
df['FIXED_REVIEW'] = df['REVIEW'] +df['TITLE']

# Simple preprocesing: lowercase, punctuation, emojis

In [7]:
df['FIXED_REVIEW'] = [review.lower() for review in df['FIXED_REVIEW']]
df['FIXED_REVIEW'] = [re.sub(r'[^a-zA-Z\s]', '', review) for review in df['FIXED_REVIEW']]
df['FIXED_REVIEW'] = [re.sub(r'@\w+', '', review) for review in df['FIXED_REVIEW']]
df['FIXED_REVIEW'] = [re.sub(r'#\w+', '', review) for review in df['FIXED_REVIEW']]
df['FIXED_REVIEW'] = [re.sub(r'http\S+|www\S+|https\S+', '', review) for review in df['FIXED_REVIEW']]
df['FIXED_REVIEW'] = [emoji.demojize(review) for review in df['FIXED_REVIEW']]

# Remove stop short and long words, lemmatize

In [8]:
stop_words = set(stopwords.words('english'))
stop_words.remove('not')
stop_words.remove('no')
def sentiment(word):
    blob = TextBlob(word)
    sentiment = blob.sentiment.polarity
    return np.abs(sentiment)

In [9]:
def remove_stop_words(text):
    text = contractions.fix(text)
    words = text.split()    # Tokenize the text into words
    lemmatizer = WordNetLemmatizer()
    filtered_words = [word for word in words if word.lower() not in stop_words]
    filtered_words = [lemmatizer.lemmatize(word) for word in filtered_words]
    filtered_words = [word for word in filtered_words if len(word) > 2]
    filtered_words = [word for word in filtered_words if len(word) < 8]
    filtered_words = [word for word in filtered_words if sentiment(word) != 0 ]
    return ' '.join(filtered_words)

In [12]:
import nltk

nltk.download('omw-1.4', quiet=True)

True

In [11]:
df['FIXED_REVIEW'] = df['FIXED_REVIEW'].apply(remove_stop_words)

# Remove misspelt words (maybe replace with better method)

In [13]:
import nltk

nltk.download('punkt_tab', quiet=True)

True

In [14]:
setofwords = set(words.words())

print("hello" in setofwords)

j = 0
new_review = []

for i in df['FIXED_REVIEW']:
    words = word_tokenize(i)
    new_words = []
    for word in words:
        if word in setofwords:
            new_words.append(word)
    sentence_correct = ' '.join(new_words)        
    new_review.append(sentence_correct)
    #df['FIXED_REVIEW'][j] = sentence_correct
    j+=1


df['FIXED_REVIEW'] = new_review

True


# Add our target (ie: class)

In [15]:
def label_race(row):
    if row['RATING'] >= 7:
        return 2
    if row['RATING'] <= 3:
        return 0
    else:
        return 1

df['CLASS'] = df.apply(label_race, axis=1) 

# Add sentiment as column (can add more)

In [16]:
def get_sentiment_1(text):
    return TextBlob(text).sentiment.polarity 
df['sentiment_1'] = df['FIXED_REVIEW'].apply(get_sentiment_1)

In [17]:
analyzer = SentimentIntensityAnalyzer()

def vader_sentiment(text):
    sentiment_score = analyzer.polarity_scores(text)
    compound_score = sentiment_score['compound']
    
    return compound_score

def vader_class(text):
    sentiment_score = analyzer.polarity_scores(text)
    compound_score = sentiment_score['compound']
    
    if compound_score > 0.1:
        sentiment = 2
    elif compound_score < -0.1:
        sentiment = 0
    else:
        sentiment = 1
    
    return sentiment

In [18]:
df['vader_sentiment'] = df['FIXED_REVIEW'].apply(vader_sentiment)
df['vader_class'] = df['FIXED_REVIEW'].apply(vader_class)

# Add a tokenized column for topics

In [19]:
def tokenize(text):
    tokens = word_tokenize(text)
    return tokens

In [20]:
df['tokenized_review'] = df['FIXED_REVIEW'].apply(tokenize)

# Now use LDA for topics, gives additional context (may need to optimise how many topics

In [21]:
dictionary = corpora.Dictionary(df['tokenized_review'])
corpus = [dictionary.doc2bow(text) for text in df['tokenized_review']]

In [22]:
lda_model = LdaModel(corpus=corpus, id2word=dictionary, num_topics=10, passes=20, alpha="auto")

In [23]:
topic_distributions = [lda_model.get_document_topics(doc) for doc in corpus]

In [24]:
topic_matrix = []
for dist in topic_distributions:
    topic_vector = [0] * 10  # Number of topics (in this case 2)
    for topic_id, prob in dist:
        topic_vector[topic_id] = prob
    topic_matrix.append(topic_vector)

# Convert topic_matrix to a DataFrame for easy handling
topic_df = pd.DataFrame(topic_matrix, columns=[f"topic_{i}" for i in range(10)])

In [25]:
df = pd.concat([df, topic_df], axis=1)
df = df.dropna()

# Now BOW with tf idf

In [26]:
v = TfidfVectorizer(ngram_range = (1,2))
bow = v.fit_transform(df.FIXED_REVIEW.values)

In [27]:
bow = pd.DataFrame.sparse.from_spmatrix(bow, columns = v.get_feature_names_out())

In [28]:
df = pd.concat([df, bow], axis=1)


In [29]:
df = df.drop(columns = ['ID','RATING','REVIEW','TITLE'])

In [30]:
df = df.dropna()

# Save data

In [ ]:
#df.to_csv('preprocessed_data.csv')

# Using the data in models

In [45]:
#Define X and y
X = df.drop(columns = ['CLASS', 'AUTHOR','FIXED_REVIEW','tokenized_review'])
y = df['CLASS']

In [46]:
X.columns

Index(['sentiment_1', 'vader_sentiment', 'vader_class', 'topic_0', 'topic_1',
       'topic_2', 'topic_3', 'topic_4', 'topic_5', 'topic_6',
       ...
       'young useless', 'young vicious', 'young victim', 'young violent',
       'young vivid', 'young whole', 'young wide', 'young willing',
       'young wrong', 'young young'],
      dtype='object', length=17268)

# Best 15 features applying correlation method

In [59]:
def correlation_feature_selection(X, y, top_k=10):
    corr_matrix = X.corrwith(y).abs().sort_values(ascending=False)
    top_features = corr_matrix.head(top_k).index.tolist()
    return X[top_features]

# Select top 15 features
X_selected = correlation_feature_selection(X, y, top_k=20)
print("Selected Features:", X_selected.columns.tolist())

Selected Features: ['vader_sentiment', 'sentiment_1', 'vader_class', 'topic_0', 'topic_6', 'bad', 'waste', 'funny', 'amazing', 'fly', 'topic_9', 'topic_7', 'perfect', 'worst', 'awful', 'great', 'rose', 'topic_4', 'really', 'past']


# We will need to iterate with different k values to find best k

# Another feature selection method is pca (this finds ten features via pca)

## Again need to find optimum k

In [60]:
from sklearn.preprocessing import StandardScaler

if isinstance(X_selected, pd.DataFrame) and X_selected.dtypes.apply(lambda x: x.name == 'Sparse[int64]').any():
    X_selected = X_selected.sparse.to_dense()

scaler = StandardScaler(with_mean=False)  
X_scaled = scaler.fit_transform(X_selected)

k = 10
pca = PCA(n_components=k)
X_pca = pca.fit_transform(X_scaled)

X_pca = pd.DataFrame(X_pca, columns=[f'PC{i+1}' for i in range(k)])

In [61]:
X_pca

,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10
0,1.904771,-0.092091,-0.086070,-0.443140,-0.260418,1.142161,0.787700,0.719729,-0.171565,0.078674
1,-0.268911,-0.768071,-0.842730,0.628006,-0.371933,1.290464,0.709102,0.407629,-0.272236,-0.311757
2,-1.566084,0.129820,-1.389244,0.947614,0.691967,0.021257,-0.694549,-0.549174,0.035403,-0.437488
3,-0.979433,-0.020297,1.420527,-0.709071,0.371252,-0.630289,-0.138900,-0.324328,-0.176987,-0.087633
4,0.314656,-0.071529,-0.070086,-2.510746,2.151323,-0.239839,-0.210476,-0.215276,0.069687,0.038133
...,...,...,...,...,...,...,...,...,...,...
5072,1.886401,-1.788384,-0.249870,-0.214195,-0.842731,-1.148162,0.148333,0.067218,-0.307993,-0.835734
5073,1.841146,-0.351336,0.021547,-2.276560,0.358392,-0.710218,0.239007,-0.034893,-0.518241,-0.025587
5074,2.655401,-1.002513,1.502082,-0.047428,-0.860252,-0.924158,-0.086556,-0.184779,-0.464846,0.044854
5075,2.376168,-0.392854,0.521726,-1.146889,-2.682512,-1.684893,0.614627,0.050976,-1.124706,-0.117780


# LASSO TBC

# Now onto our models

In [62]:
X_train, X_test, y_train, y_test = train_test_split(X_pca, y, test_size=0.2, random_state=51, stratify=y)

# Multinomial Logisitc Regression

### Initial Model

In [64]:
model_mlr = LogisticRegression(multi_class='multinomial', solver='lbfgs',
                                   class_weight='balanced', max_iter=1000)

model_mlr.fit(X_train, y_train)

y_pred = model_mlr.predict(X_test)
print("Initial Classification Report:")
print(classification_report(y_test, y_pred, zero_division=1))

Initial Classification Report:
              precision    recall  f1-score   support

         0.0       0.70      0.66      0.68       388
         1.0       0.21      0.15      0.17       211
         2.0       0.63      0.76      0.69       417

    accuracy                           0.59      1016
   macro avg       0.51      0.52      0.51      1016
weighted avg       0.57      0.59      0.58      1016



### MLR - Hyperparameter Tuning with GridSearchCV

In [75]:
param_grid = {
    'C': [0.00001, 0.0001, 0.001, 0.01, 0.1, 1, 10],
    'solver': ['lbfgs', 'saga'],
}

grid = GridSearchCV(
    LogisticRegression(multi_class='multinomial', class_weight='balanced', max_iter=1000),
    param_grid,
    scoring='accuracy',
    cv=5,
    n_jobs=-1
)

grid.fit(X_train, y_train)

print("\nBest Parameters:", grid.best_params_)

y_pred_tuned = grid.predict(X_test)
print("\nTuned Classification Report:")
print(classification_report(y_test, y_pred_tuned, zero_division=1))

<IPython.core.display.Javascript object>


Best Parameters: {'C': 1e-05, 'solver': 'lbfgs'}

Tuned Classification Report:
              precision    recall  f1-score   support

         0.0       0.68      0.72      0.70       388
         1.0       0.24      0.02      0.04       211
         2.0       0.60      0.84      0.70       417

    accuracy                           0.62      1016
   macro avg       0.51      0.53      0.48      1016
weighted avg       0.56      0.62      0.56      1016



# Random Forest

### Initial Model

In [76]:
model_rf = RandomForestClassifier(random_state=51, class_weight='balanced')

model_rf.fit(X_train, y_train)
y_pred = model_rf.predict(X_test)

print("\nInitial Classification Report:")
print(classification_report(y_test, y_pred, zero_division=1))


Initial Classification Report:
              precision    recall  f1-score   support

         0.0       0.62      0.69      0.65       388
         1.0       0.16      0.08      0.10       211
         2.0       0.61      0.71      0.65       417

    accuracy                           0.57      1016
   macro avg       0.46      0.49      0.47      1016
weighted avg       0.52      0.57      0.54      1016



### RF - Hyperparameter Tuning with RandomizedSearchCV

In [77]:
param_dist = {
    'n_estimators': randint(50, 500),
    'max_depth': randint(1, 20),
    'min_samples_split': randint(2, 10),
    'min_samples_leaf': randint(1, 10),
    'max_features': ['auto', 'sqrt', 'log2']
}

rand_search = RandomizedSearchCV(
    estimator=model_rf,
    param_distributions=param_dist,
    n_iter=10,
    cv=5,
    scoring='accuracy',
    n_jobs=-1,
    random_state=27
)

rand_search.fit(X_train, y_train)

best_rf = rand_search.best_estimator_

y_pred_best = best_rf.predict(X_test)
accuracy_best = accuracy_score(y_test, y_pred_best)

print("\nBest Hyperparameters:", rand_search.best_params_)

print("\nTuned RF Classification Report:")
print(classification_report(y_test, y_pred_best, zero_division=1))


Best Hyperparameters: {'max_depth': 3, 'max_features': 'auto', 'min_samples_leaf': 4, 'min_samples_split': 5, 'n_estimators': 183}

Tuned RF Classification Report:
              precision    recall  f1-score   support

         0.0       0.65      0.78      0.71       388
         1.0       0.31      0.07      0.12       211
         2.0       0.64      0.78      0.70       417

    accuracy                           0.63      1016
   macro avg       0.53      0.54      0.51      1016
weighted avg       0.58      0.63      0.58      1016



# Support Vector Machine

### Initial Model

In [78]:
model_svm = svm.SVC(kernel='linear', class_weight='balanced')
model_svm.fit(X_train, y_train)

y_pred = model_svm.predict(X_test)
print("\nInitial Classification Report:")
print(classification_report(y_test, y_pred, zero_division=1))


Initial Classification Report:
              precision    recall  f1-score   support

         0.0       0.70      0.65      0.68       388
         1.0       0.21      0.08      0.11       211
         2.0       0.61      0.84      0.71       417

    accuracy                           0.61      1016
   macro avg       0.50      0.52      0.50      1016
weighted avg       0.56      0.61      0.57      1016



### SVM - RandomizedSearchCV for Hyperparameter Tuning

In [79]:
from scipy.stats import uniform

param_dist = {
    'C': uniform(0.01, 10),
    'kernel': ['linear', 'rbf', 'poly', 'sigmoid'],
    'gamma': ['scale', 'auto'],
    'degree': [2, 3, 4],
}

rand_search = RandomizedSearchCV(
    estimator=svm.SVC(class_weight='balanced'),
    param_distributions=param_dist,
    n_iter=10,
    cv=5,
    scoring='accuracy',
    n_jobs=-1,
    random_state=27
)

rand_search.fit(X_train, y_train)

best_svm = rand_search.best_estimator_
y_pred_best = best_svm.predict(X_test)

print("\nBest Hyperparameters:", rand_search.best_params_)
print("\nTuned Classification Report:")
print(classification_report(y_test, y_pred_best, zero_division=1))


Best Hyperparameters: {'C': 4.267214105188957, 'degree': 2, 'gamma': 'auto', 'kernel': 'linear'}

Tuned Classification Report:
              precision    recall  f1-score   support

         0.0       0.70      0.65      0.68       388
         1.0       0.21      0.08      0.11       211
         2.0       0.61      0.84      0.70       417

    accuracy                           0.61      1016
   macro avg       0.51      0.52      0.50      1016
weighted avg       0.56      0.61      0.57      1016



# XGBoost

### Initial Model

In [80]:
from xgboost import XGBClassifier

model_xgb = XGBClassifier(objective='multi:softmax', num_class=3, eval_metric='mlogloss')
model_xgb.fit(X_train, y_train)

y_pred = model_xgb.predict(X_test)
print("\nInitial Classification Report:")
print(classification_report(y_test, y_pred, zero_division=1))


Initial Classification Report:
              precision    recall  f1-score   support

         0.0       0.63      0.72      0.67       388
         1.0       0.23      0.09      0.13       211
         2.0       0.63      0.74      0.68       417

    accuracy                           0.60      1016
   macro avg       0.50      0.52      0.49      1016
weighted avg       0.55      0.60      0.56      1016



### XGBoost - RandomizedSearchCV for Hyperparameter Tuning

In [81]:
from scipy.stats import randint, uniform

param_dist = {
    'n_estimators': randint(50, 500),       
    'max_depth': randint(3, 15),      
    'learning_rate': uniform(0.01, 0.3),   
    'subsample': uniform(0.5, 0.5),      
    'colsample_bytree': uniform(0.5, 0.5), 
    'gamma': uniform(0, 5),       
    'reg_alpha': uniform(0, 1),  
    'reg_lambda': uniform(1, 5)   
}

rand_search = RandomizedSearchCV(
    estimator=XGBClassifier(objective='multi:softmax', num_class=3, eval_metric='mlogloss', use_label_encoder=False),
    param_distributions=param_dist,
    n_iter=10,              
    cv=5,                
    scoring='accuracy',  
    n_jobs=-1,      
    random_state=27
)

rand_search.fit(X_train, y_train)

best_xgb = rand_search.best_estimator_
y_pred_best = best_xgb.predict(X_test)

print("\nBest Hyperparameters:", rand_search.best_params_)
print("\nTuned Classification Report:")
print(classification_report(y_test, y_pred_best, zero_division=1))


Best Hyperparameters: {'colsample_bytree': 0.7128607052594479, 'gamma': 4.072918702472763, 'learning_rate': 0.2306191870599039, 'max_depth': 8, 'n_estimators': 74, 'reg_alpha': 0.3833807728726202, 'reg_lambda': 5.897283160722603, 'subsample': 0.9465971733575677}

Tuned Classification Report:
              precision    recall  f1-score   support

         0.0       0.64      0.77      0.70       388
         1.0       0.22      0.02      0.03       211
         2.0       0.62      0.79      0.70       417

    accuracy                           0.62      1016
   macro avg       0.49      0.53      0.48      1016
weighted avg       0.55      0.62      0.56      1016

